### evaluating the output forecastings

output of forecastings format:

    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign	endlap	pred_endlap
      11	12	31	3.0	5.0	2.0	1	1.0	-2.0	-1	49	58
    
refer to:
    19.RankNet/stage_model_regressor.ipynb
    19.RankNet/RankForecasting-stint-paper-1kpitmodel.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [ ]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

In [37]:
def do_rerank(dfout, short=True):
    """
    carno','startlap','startrank','endrank','diff','sign','pred_endrank','pred_diff','pred_sign','endlap','pred_endlap
    
    output of prediction of target can be float
    
    resort the endrank globally
    
    """
    
    cols=['carno','startlap','startrank','endrank','diff','sign','pred_endrank','pred_diff','pred_sign','endlap','pred_endlap']
    colid={x:id for id,x in enumerate(cols)}
    
    #df = dfout.sort_values(by=['startlap','carno'])
    print('rerank...')
    laps = set(dfout.startlap.values)
    
    dfs = []
    for lap in laps:
        df = dfout[dfout['startlap']==lap].to_numpy()
        
        #print('in',df)
        
        idx = np.argsort(df[:,colid['pred_endrank']], axis=0)
        true_rank = np.argsort(idx, axis=0)
    
        df[:,colid['pred_endrank']] = true_rank
        
        #reset preds 
        df[:,colid['pred_diff']] = df[:,colid['pred_endrank']] - df[:,colid['endrank']]

        for rec in df:
            if rec[colid['pred_diff']] == 0:
                rec[colid['pred_sign']] = 0
            elif rec[colid['pred_diff']] > 0:
                rec[colid['pred_sign']] = 1
            else:
                rec[colid['pred_sign']] = -1        
        
        #print('out',df)
        if len(dfs) == 0:
            dfs = df
        else:
            dfs = np.vstack((dfs, df))
        #dfs.append(df)
        #np.vstack(df)
        
    #dfret = pd.concat(dfs)
    #data = np.array(dfs)
    if short:
        dfret = pd.DataFrame(dfs.astype(int), columns = cols[:-2])
    else:
        dfret = pd.DataFrame(dfs.astype(int), columns = cols)
    return dfret

In [2]:
def load_dfout(datafile):
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout[0]

In [3]:
### load ml models
_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
preddf = load_dfout(outfile)
outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-oracle-t{_trim}.pickle'
preddf_oracle = load_dfout(outfile)

In [4]:
dfout = preddf['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)


model: currank
model: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.00}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: avgrank
model: acc={0.38}, mae={17.43}, rmse={19.94},r2={-4.83}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: dice
model: acc={0.38}, mae={6.56}, rmse={8.57},r2={-0.94}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: lasso
model: acc={0.67}, mae={3.29}, rmse={4.51},r2={-1.47}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: ridge
model: acc={0.62}, mae={3.64}, rmse={4.87},r2={-0.88}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: rf
model: acc={0.62}, mae={3.35}, rmse={4.53},r2={-1.72}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: svr_rbf
model: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.00}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: svr_lin
model: acc={0.61}, mae={3.42}, rmse={4.78},r2

In [7]:
df = preddf['2018']['svr_lin']
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
39,12,31,3,5,2,1,3,0,0
40,12,49,5,0,-5,-1,1,-4,-1
41,12,93,0,1,1,1,-1,-1,-1
42,12,128,1,7,6,1,-1,-2,-1
43,12,170,7,0,-7,-1,3,-4,-1


In [8]:
dfout = preddf['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

model: currank
model: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.00}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: avgrank
model: acc={0.31}, mae={17.49}, rmse={19.56},r2={-5.53}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: dice
model: acc={0.48}, mae={6.20}, rmse={8.64},r2={-1.08}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: lasso
model: acc={0.57}, mae={4.03}, rmse={5.45},r2={-1.33}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: ridge
model: acc={0.56}, mae={4.29}, rmse={5.81},r2={-0.79}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: rf
model: acc={0.56}, mae={4.06}, rmse={5.66},r2={-0.92}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: svr_rbf
model: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.00}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}
model: svr_lin
model: acc={0.57}, mae={4.72}, rmse={6.41},r2

In [9]:
# load ranknet results
outfile=f'stint-dfout-ranknet-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
ranknetdf = load_dfout(outfile)

In [10]:
dfout = ranknetdf['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

model: pitmodel_mode
model: acc={0.58}, mae={3.82}, rmse={5.48},r2={-3.97}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}
model: pitmodel_mean
model: acc={0.65}, mae={3.80}, rmse={5.43},r2={-4.84}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}
model: pitmodel_median
model: acc={0.56}, mae={3.85}, rmse={5.49},r2={-4.29}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}
model: oracle_mode
model: acc={0.67}, mae={3.46}, rmse={5.11},r2={-3.21}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}
model: oracle_mean
model: acc={0.67}, mae={3.46}, rmse={5.11},r2={-3.21}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}
model: oracle_median
model: acc={0.67}, mae={3.46}, rmse={5.11},r2={-3.21}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.71},r2={0.53}


In [11]:
dfout = ranknetdf['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

model: pitmodel_mode
model: acc={0.44}, mae={3.99}, rmse={6.07},r2={-2.04}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}
model: pitmodel_mean
model: acc={0.50}, mae={3.91}, rmse={5.94},r2={-2.07}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}
model: pitmodel_median
model: acc={0.45}, mae={3.95}, rmse={6.01},r2={-2.05}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}
model: oracle_mode
model: acc={0.50}, mae={3.81}, rmse={5.79},r2={-1.77}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}
model: oracle_mean
model: acc={0.50}, mae={3.81}, rmse={5.79},r2={-1.77}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}
model: oracle_median
model: acc={0.50}, mae={3.81}, rmse={5.79},r2={-1.77}, 
            naive: acc={0.18}, mae={4.04}, rmse={6.25},r2={0.46}


In [43]:
ranknetdf['2018']['pitmodel_mean']

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
0,1.0,100.0,8.0,7.0,-1.0,-1.0,6.28,-1.72,-1.0,134.0,128.0
1,1.0,69.0,8.0,8.0,0.0,0.0,5.03,-2.97,-1.0,100.0,101.0
2,1.0,134.0,7.0,16.0,9.0,1.0,6.78,-0.22,-1.0,169.0,166.0
3,1.0,169.0,16.0,7.0,-9.0,-1.0,14.67,-1.33,-1.0,199.0,199.0
4,1.0,49.0,2.0,8.0,6.0,1.0,1.76,-0.24,-1.0,69.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...
183,98.0,169.0,13.0,11.0,-2.0,-1.0,10.40,-2.60,-1.0,199.0,199.0
184,98.0,49.0,9.0,14.0,5.0,1.0,8.84,-0.16,-1.0,91.0,79.0
185,98.0,91.0,14.0,9.0,-5.0,-1.0,11.02,-2.98,-1.0,124.0,116.0
186,98.0,124.0,9.0,13.0,4.0,1.0,5.27,-3.73,-1.0,169.0,150.0


In [44]:
### stint result, just cannot do rerank, 
### instead use int()
df = do_rerank(ranknetdf['2018']['pitmodel_mean'],short=False)
stint.get_evalret(df)

rerank...


In [46]:
df[df['startlap']]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
0,12,128,1,7,6,1,0,-7,-1,170,156
1,21,128,25,22,-3,-1,2,-20,-1,176,155
2,59,128,22,21,-1,-1,1,-20,-1,155,157
3,9,129,8,10,2,1,1,-9,-1,159,158
4,14,129,19,21,2,1,3,-18,-1,142,167
...,...,...,...,...,...,...,...,...,...,...,...
183,98,124,9,13,4,1,0,-13,-1,169,150
184,4,126,14,13,-1,-1,1,-12,-1,171,155
185,20,126,4,3,-1,-1,0,-3,-1,172,158
186,3,127,4,26,22,1,1,-25,-1,145,145


### sync with the dataset

In [12]:
def eval_bydf_v0(testdf, bydf, force2int=False):
    #collect only records in bydf <carno and startlap>
    cars = set(bydf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(bydf[bydf['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:    
            dfrec = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)]
            
            retdf.append(dfrec)
        
    dfout = pd.concat(retdf)
    
    if force2int:
        dfdata = dfout.to_numpy().astype(int)
        dfout = pd.DataFrame(dfdata, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         'endlap','pred_endlap'
                                        ])
    
    print('df size:', len(dfout))
    stint.get_evalret(dfout)
    return dfout  


def eval_bydf(testdf, bydf, forcematch=True, force2int=False):
    #collect only records in bydf <carno and startlap>
    cars = set(bydf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(bydf[bydf['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:    
            dfrec = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)]
            
            #check match
            if forcematch:
                a = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)].to_numpy().astype(int)
                b = bydf[(bydf['carno']==car) & (bydf['startlap']==startlap)].to_numpy().astype(int)

                if len(a)!=0 and len(b)!=0:
                    #compare 
                    #startrank, endrank
                    if not ((a[0][2] == b[0][2]) and (a[0][3] == b[0][3])):
                        #print('mismatch:', a, b)            
                        continue
            
            retdf.append(dfrec)
        
    dfout = pd.concat(retdf)
    
    if force2int:
        dfdata = dfout.to_numpy().astype(int)
        dfout = pd.DataFrame(dfdata, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         'endlap','pred_endlap'
                                        ])
    
    dfout = dfout.sort_values(by=['carno','startlap'])
    
    print('df size:', len(dfout))
    stint.get_evalret(dfout)
    return dfout  

def cmp_df(testdf, bydf):
    #collect only records in bydf <carno and startlap>
    cars = set(bydf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(bydf[bydf['carno']==car].startlap.values)
        
    retdf = []
    errcnt = 0
    for car in cars:
        for startlap in startlaps[car]:    
            a = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)].to_numpy().astype(int)
            b = bydf[(bydf['carno']==car) & (bydf['startlap']==startlap)].to_numpy().astype(int)
            
            if len(a)!=0 and len(b)!=0:
                #compare 
                #startrank, endrank
                if not ((a[0][2] == b[0][2]) and (a[0][3] == b[0][3])):
                    print('mismatch:', a, b)
                errcnt += 1
    print('errcnt:', errcnt)
    return errcnt

In [13]:
cmp_df(ranknetdf['2018']['pitmodel_mean'], preddf['2018']['lasso'])

mismatch: [[  3 127   4  26  22   1   2  -1  -1 145 145]] [[  3 127   4  25  21   1   6   2   1]]
mismatch: [[  6 136  14  10  -4  -1  12  -1  -1 179 168]] [[  6 136  15  10  -5  -1  10  -5  -1]]
mismatch: [[ 17 135  24  25   1   1  22  -1  -1 139 165]] [[ 17 135  25  25   0   0  25   0   0]]
mismatch: [[ 17 186  19  19   0   0  18   0  -1 190 198]] [[ 17 186  20  19  -1  -1  22   2   1]]
mismatch: [[ 21 189  21  19  -2  -1  18  -2  -1 198 198]] [[ 21 189  21  21   0   0  21   0   0]]
mismatch: [[ 26 100  23  14  -9  -1  25   2   1 132 129]] [[ 26 100  23  25   2   1  20  -3  -1]]
mismatch: [[ 26  69  26   9 -17  -1  25   0  -1  99  97]] [[26 69 26 23 -3 -1 23 -3 -1]]
mismatch: [[26 52 27 25 -2 -1 25 -1 -1 68 80]] [[26 52 27 29  2  1 28  1  1]]
mismatch: [[ 26 155  23  21  -2  -1  21  -1  -1 178 183]] [[ 26 155  23  22  -1  -1  20  -3  -1]]
mismatch: [[32 33 31 30 -1 -1 25 -5 -1 50 64]] [[32 33 31 32  1  1 26 -5 -1]]
mismatch: [[ 32  97  27  27   0   0  27   0   0 108 109]] [[32 97 27 

185

In [15]:
for year in ['2018','2019']:
    for clf in ['rf','xgb','svr_lin']:
        #eval_bydf(ranknetdf['2018']['pitmodel_mean'], preddf['2018']['lasso'],force2int=True)
        #eval_bydf(preddf['2018']['lasso'],ranknetdf['2018']['pitmodel_mean'])
        print('year:',year,'clf:',clf)
        eval_bydf(ranknetdf[year]['pitmodel_mean'], preddf[year][clf],force2int=True)
        eval_bydf(preddf[year][clf],ranknetdf[year]['pitmodel_mean'])


year: 2018 clf: rf
df size: 172
model: acc={0.66}, mae={3.73}, rmse={5.12},r2={-5.49}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
df size: 172
model: acc={0.62}, mae={3.41}, rmse={4.50},r2={-1.57}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
year: 2018 clf: xgb
df size: 172
model: acc={0.66}, mae={3.73}, rmse={5.12},r2={-5.49}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
df size: 172
model: acc={0.59}, mae={3.63}, rmse={4.71},r2={-0.35}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
year: 2018 clf: svr_lin
df size: 172
model: acc={0.66}, mae={3.73}, rmse={5.12},r2={-5.49}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
df size: 172
model: acc={0.62}, mae={3.48}, rmse={4.72},r2={-1.39}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}
year: 2019 clf: rf
df size: 201
model: acc={0.51}, mae={3.86}, rmse={5.97},r2={-2.69}, 
            naive: acc={0.16}, mae={4.22}, rm

In [47]:
eval_bydf(ranknetdf['2018']['pitmodel_mean'], preddf['2018']['lasso'],force2int=True)

df size: 172
model: acc={0.66}, mae={3.73}, rmse={5.12},r2={-5.49}, {172}
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
5,1,31,2,2,0,0,2,0,1,49,53
4,1,49,2,8,6,1,1,0,-1,69,78
1,1,69,8,8,0,0,5,-2,-1,100,101
0,1,100,8,7,-1,-1,6,-1,-1,134,128
2,1,134,7,16,9,1,6,0,-1,169,166
...,...,...,...,...,...,...,...,...,...,...,...
171,98,31,6,9,3,1,6,0,1,49,56
168,98,49,9,14,5,1,8,0,-1,91,79
169,98,91,14,9,-5,-1,11,-2,-1,124,116
170,98,124,9,13,4,1,5,-3,-1,169,150


In [95]:
eval_bydf(preddf['2018']['lasso'],ranknetdf['2018']['pitmodel_mean'])

df size: 172
model: acc={0.67}, mae={3.33}, rmse={4.43},r2={-1.32}, 
            naive: acc={0.10}, mae={4.23}, rmse={5.51},r2={0.56}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,31,2,2,0,0,6,4,1
1,1,49,2,8,6,1,5,3,1
2,1,69,8,8,0,0,7,-1,-1
3,1,100,8,7,-1,-1,8,0,0
4,1,134,7,16,9,1,7,0,0
...,...,...,...,...,...,...,...,...,...
184,98,31,6,9,3,1,11,5,1
185,98,49,9,14,5,1,8,-1,-1
186,98,91,14,9,-5,-1,12,-2,-1
187,98,124,9,13,4,1,7,-2,-1


In [42]:
eval_bydf(ranknetdf['2018']['pitmodel_mean'], preddf['2018']['lasso'],force2int=True)

df size: 185
model: acc={0.66}, mae={3.82}, rmse={5.45},r2={-6.57}, 
            naive: acc={0.10}, mae={4.29}, rmse={5.75},r2={0.53}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
0,1,100,8,7,-1,-1,6,-1,-1,134,128
1,1,69,8,8,0,0,5,-2,-1,100,101
2,1,134,7,16,9,1,6,0,-1,169,166
3,1,169,16,7,-9,-1,14,-1,-1,199,199
4,1,49,2,8,6,1,1,0,-1,69,78
...,...,...,...,...,...,...,...,...,...,...,...
180,98,169,13,11,-2,-1,10,-2,-1,199,199
181,98,49,9,14,5,1,8,0,-1,91,79
182,98,91,14,9,-5,-1,11,-2,-1,124,116
183,98,124,9,13,4,1,5,-3,-1,169,150


In [41]:
eval_bydf(preddf['2018']['lasso'],ranknetdf['2018']['pitmodel_mean'])

df size: 185
model: acc={0.66}, mae={3.34}, rmse={4.56},r2={-1.48}, 
            naive: acc={0.10}, mae={4.16}, rmse={5.56},r2={0.56}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
3,1,100,8,7,-1,-1,8,0,0
2,1,69,8,8,0,0,7,-1,-1
4,1,134,7,16,9,1,7,0,0
5,1,169,16,7,-9,-1,11,-5,-1
1,1,49,2,8,6,1,5,3,1
...,...,...,...,...,...,...,...,...,...
188,98,169,13,11,-2,-1,11,-2,-1
185,98,49,9,14,5,1,8,-1,-1
186,98,91,14,9,-5,-1,12,-2,-1
187,98,124,9,13,4,1,7,-2,-1


In [36]:
eval_bydf(ranknetdf['2019']['pitmodel_mean'], preddf['2019']['lasso'])

df size: 213
model: acc={0.51}, mae={3.98}, rmse={6.03},r2={-2.18}, 
            naive: acc={0.17}, mae={4.17}, rmse={6.38},r2={0.44}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
0,2.0,65.0,18.0,17.0,-1.0,-1.0,8.40,-9.60,-1.0,101.0,91.0
1,2.0,33.0,27.0,18.0,-9.0,-1.0,9.80,-17.20,-1.0,65.0,66.0
2,2.0,101.0,17.0,13.0,-4.0,-1.0,6.52,-10.48,-1.0,134.0,127.0
3,2.0,134.0,13.0,8.0,-5.0,-1.0,6.07,-6.93,-1.0,170.0,163.0
4,2.0,170.0,8.0,4.0,-4.0,-1.0,2.54,-5.46,-1.0,179.0,196.0
...,...,...,...,...,...,...,...,...,...,...,...
216,98.0,176.0,25.0,25.0,0.0,0.0,24.73,-0.27,-1.0,178.0,194.0
218,98.0,178.0,25.0,25.0,0.0,0.0,25.00,0.00,0.0,180.0,194.0
219,98.0,179.0,25.0,25.0,0.0,0.0,25.00,0.00,0.0,180.0,194.0
220,98.0,180.0,25.0,25.0,0.0,0.0,25.00,0.00,0.0,194.0,194.0


In [92]:
eval_bydf(ranknetdf['2019']['pitmodel_mean'], preddf['2019']['lasso'],force2int=True)

df size: 201
model: acc={0.51}, mae={3.86}, rmse={5.97},r2={-2.69}, 
            naive: acc={0.16}, mae={4.22}, rmse={6.41},r2={0.45}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
1,2,33,27,18,-9,-1,9,-17,-1,65,66
0,2,65,18,17,-1,-1,8,-9,-1,101,91
2,2,101,17,13,-4,-1,6,-10,-1,134,127
3,2,134,13,8,-5,-1,6,-6,-1,170,163
4,2,170,8,4,-4,-1,2,-5,-1,179,196
...,...,...,...,...,...,...,...,...,...,...,...
194,98,169,29,29,0,0,21,-7,-1,174,194
196,98,176,25,25,0,0,24,0,-1,178,194
197,98,178,25,25,0,0,25,0,0,180,194
198,98,179,25,25,0,0,25,0,0,180,194


In [91]:
eval_bydf(preddf['2019']['lasso'],ranknetdf['2019']['pitmodel_mean'])

df size: 201
model: acc={0.57}, mae={3.99}, rmse={5.45},r2={-1.36}, 
            naive: acc={0.16}, mae={4.22}, rmse={6.41},r2={0.45}


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,2,33,27,18,-9,-1,15,-12,-1
1,2,65,18,17,-1,-1,14,-4,-1
2,2,101,17,13,-4,-1,12,-5,-1
3,2,134,13,8,-5,-1,10,-3,-1
4,2,170,8,4,-4,-1,10,2,1
...,...,...,...,...,...,...,...,...,...
210,98,169,29,29,0,0,28,-1,-1
212,98,176,25,25,0,0,26,1,1
213,98,178,25,25,0,0,25,0,0
214,98,179,25,25,0,0,25,0,0


In [89]:
df1= eval_bydf(ranknetdf['2019']['pitmodel_mean'], preddf['2019']['lasso'],force2int=True)

df size: 201
model: acc={0.51}, mae={3.86}, rmse={5.97},r2={-2.69}, 
            naive: acc={0.16}, mae={4.22}, rmse={6.41},r2={0.45}


In [90]:
df2=eval_bydf(preddf['2019']['lasso'],ranknetdf['2019']['pitmodel_mean'])

df size: 201
model: acc={0.57}, mae={3.99}, rmse={5.45},r2={-1.36}, 
            naive: acc={0.16}, mae={4.22}, rmse={6.41},r2={0.45}


In [39]:
stint.get_evalret(preddf['2019']['lasso'])

model: acc={0.57}, mae={4.03}, rmse={5.45},r2={-1.33}, 
            naive: acc={0.15}, mae={4.29}, rmse={6.46},r2={0.44}


array([[ 0.56944444,  4.02777778,  5.4526582 , -1.32859663],
       [ 0.15277778,  4.29166667,  6.46034915,  0.44268267]])

In [82]:
a = df1['diff'].values
b = df2['diff'].values
a==b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [84]:
a = df1['startrank'].values
b = df2['startrank'].values
np.where(a!=b)

(array([], dtype=int64),)

In [85]:
a = df1['endrank'].values
b = df2['endrank'].values
np.where(a!=b)

(array([ 34,  60,  61,  62,  91, 180, 187, 188]),)

In [86]:
df1.head(35)


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign,endlap,pred_endlap
1,2,33,27,18,-9,-1,9,-17,-1,65,66
0,2,65,18,17,-1,-1,8,-9,-1,101,91
2,2,101,17,13,-4,-1,6,-10,-1,134,127
3,2,134,13,8,-5,-1,6,-6,-1,170,163
4,2,170,8,4,-4,-1,2,-5,-1,179,196
5,2,179,4,3,-1,-1,1,-2,-1,199,199
6,3,35,7,29,22,1,6,0,-1,40,66
8,3,40,29,27,-2,-1,26,-2,-1,68,70
7,3,68,27,27,0,0,26,0,-1,75,98
9,3,75,27,26,-1,-1,27,0,1,109,104


In [87]:
df2.head(35)

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,2,33,27,18,-9,-1,15,-12,-1
1,2,65,18,17,-1,-1,14,-4,-1
2,2,101,17,13,-4,-1,12,-5,-1
3,2,134,13,8,-5,-1,10,-3,-1
4,2,170,8,4,-4,-1,10,2,1
5,2,179,4,3,-1,-1,5,1,1
6,3,35,7,29,22,1,13,6,1
7,3,40,29,27,-2,-1,20,-9,-1
8,3,68,27,27,0,0,27,0,0
9,3,75,27,26,-1,-1,21,-6,-1


### shortterm test results

In [54]:
outfile=f'shortterm-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-rerank-t{_trim}.pickle'
shortterm_df = load_dfout(outfile)

In [55]:
dfout = shortterm_df['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret_shortterm(df)

model: currank
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: avgrank
model: acc={0.64}, mae={2.04}, rmse={3.59},r2={0.82}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: dice
model: acc={0.45}, mae={2.21}, rmse={4.01},r2={0.78}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: lasso
model: acc={0.73}, mae={1.41}, rmse={3.22},r2={0.85}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: ridge
model: acc={0.73}, mae={1.42}, rmse={3.22},r2={0.85}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: rf
model: acc={0.51}, mae={1.75}, rmse={3.34},r2={0.84}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: svr
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}
model: xgb
model: acc={0.46}, mae={1

In [56]:
dfout = shortterm_df['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret_shortterm(df)

model: currank
model: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: avgrank
model: acc={0.60}, mae={1.81}, rmse={3.36},r2={0.85}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: dice
model: acc={0.49}, mae={2.06}, rmse={3.83},r2={0.81}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: lasso
model: acc={0.74}, mae={1.24}, rmse={3.04},r2={0.88}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: ridge
model: acc={0.74}, mae={1.24}, rmse={3.04},r2={0.88}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: rf
model: acc={0.62}, mae={1.33}, rmse={2.88},r2={0.89}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: svr
model: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}, {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}
model: xgb
model: acc={0.64}, mae={1

In [24]:
np.sum(df['diff'].values == 0)/len(df)

0.5906367041198501

In [22]:
outfile=f'shortterm-dfout-ranknet-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
sranknet_df = load_dfout(outfile)

In [23]:
sranknet_df.keys()

dict_keys(['2018', '2019'])

In [17]:
sranknet_df['2018']['oracle-TIMEDIFF-2018-inlap-nopitage']

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,10,3.0,3.0,0.0,0,3.0,0.0,0
1,3,10,7.0,7.0,0.0,0,7.0,0.0,0
2,4,10,12.0,12.0,0.0,0,12.0,0.0,0
3,6,10,15.0,15.0,0.0,0,15.0,0.0,0
4,7,10,28.0,28.0,0.0,0,28.0,0.0,0
...,...,...,...,...,...,...,...,...,...
5335,60,197,15.0,15.0,0.0,0,16.0,1.0,1
5336,64,197,16.0,16.0,0.0,0,14.0,-2.0,-1
5337,66,197,10.0,10.0,0.0,0,10.0,0.0,0
5338,88,197,13.0,13.0,0.0,0,13.0,0.0,0


In [34]:
dfout = sranknet_df['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret_shortterm(df)

model: oracle-TIMEDIFF-2018-inlap-nopitage
model: acc={0.84}, mae={0.89}, rmse={2.13},r2={0.94}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}


In [24]:
dfout = sranknet_df['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret_shortterm(df)

model: oracle-TIMEDIFF-2019-inlap-nopitage
model: acc={0.81}, mae={0.84}, rmse={2.11},r2={0.94}, {188}
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}


In [19]:
dfx = sranknet_df['2018']['oracle-TIMEDIFF-2018-inlap-nopitage']

In [23]:
np.sum(dfx['diff'].values == 0)/len(dfx)

0.5906367041198501

In [35]:
dfout = sranknet_df['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret_shortterm(df)

model: oracle-TIMEDIFF-2019-inlap-nopitage
model: acc={0.81}, mae={0.84}, rmse={2.11},r2={0.94}, {188}
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}


In [63]:
df = shortterm_df['2018']['lasso']
df[df['carno']==12].head(40)

,carno,startlap,startrank,endrank,pred_endrank,diff,sign,pred_diff,pred_sign
7,12.0,9.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
40,12.0,10.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
73,12.0,11.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
106,12.0,12.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
139,12.0,13.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
172,12.0,14.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
205,12.0,15.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
238,12.0,16.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
271,12.0,17.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
304,12.0,18.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0


In [61]:
np.sum(df.sign ==df.pred_sign)/len(df)

0.5820224719101124

In [25]:
outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-usemean.pickle'
ranknet_mlp = load_dfout(outfile)

In [26]:
df_ranknet_mlp_2018 = ranknet_mlp['2018']['pitmodel_mean']

In [27]:
df_ranknet_mlp_2018

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1.0,10.0,3.0,3.0,0.0,0.0,3.00,0.00,0.0
1,1.0,11.0,3.0,3.0,0.0,0.0,3.00,0.00,0.0
2,1.0,12.0,3.0,3.0,0.0,0.0,3.00,0.00,0.0
3,1.0,13.0,3.0,3.0,0.0,0.0,3.00,0.00,0.0
4,1.0,14.0,3.0,3.0,0.0,0.0,3.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...
5335,98.0,193.0,15.0,13.0,-2.0,-1.0,15.02,0.02,1.0
5336,98.0,194.0,15.0,11.0,-4.0,-1.0,14.99,-0.01,-1.0
5337,98.0,195.0,13.0,11.0,-2.0,-1.0,12.98,-0.02,-1.0
5338,98.0,196.0,11.0,11.0,0.0,0.0,10.97,-0.03,-1.0


In [38]:
df = do_rerank(df_ranknet_mlp_2018)
df

rerank...


,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,10,3,3,0,0,3,0,0
1,3,10,7,7,0,0,7,0,0
2,4,10,12,12,0,0,12,0,0
3,6,10,15,15,0,0,15,0,0
4,7,10,28,28,0,0,28,0,0
...,...,...,...,...,...,...,...,...,...
5335,60,197,15,15,0,0,15,0,0
5336,64,197,16,16,0,0,16,0,0
5337,66,197,10,10,0,0,10,0,0
5338,88,197,13,13,0,0,13,0,0


In [39]:
stint.get_evalret_shortterm(df)

model: acc={0.78}, mae={1.21}, rmse={2.87},r2={0.88}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}


array([[0.78191489, 1.21385768, 2.87009637, 0.88499164],
       [0.71808511, 1.33932584, 3.24896987, 0.85262365]])

In [40]:
df_ranknet_mlp_2019 = do_rerank(ranknet_mlp['2019']['pitmodel_mean'])

rerank...


In [42]:
df_ranknet_mlp_2019

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,2,10,4,4,0,0,4,0,0
1,3,10,11,11,0,0,11,0,0
2,4,10,21,21,0,0,21,0,0
3,5,10,25,25,0,0,25,0,0
4,7,10,12,12,0,0,12,0,0
...,...,...,...,...,...,...,...,...,...
5624,28,197,6,7,1,1,6,-1,-1
5625,30,197,2,2,0,0,2,0,0
5626,33,197,11,11,0,0,11,0,0
5627,39,197,15,15,0,0,15,0,0


In [41]:
stint.get_evalret_shortterm(df_ranknet_mlp_2019)

model: acc={0.78}, mae={1.07}, rmse={2.74},r2={0.90}, {188}
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}


array([[0.77659574, 1.07408065, 2.73914791, 0.90201302],
       [0.73404255, 1.16255107, 3.05012312, 0.87850116]])